In [40]:
import re

In [1]:
from qdrant_client import QdrantClient

/Users/huynhanhkiet/Desktop/Fine-tune-LegalVN/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
collection_name = "thue-phi-le-phi_all-MiniLM-L6-v2"

# Initialize Qdrant client
client = QdrantClient(
    host="localhost", 
    port=6333, 
    timeout=600.0
)

In [9]:
# Lấy toàn bộ points bằng scroll
all_points = []
scroll_filter = None
offset = None

while True:
    scroll_result = client.scroll(
        collection_name=collection_name,
        scroll_filter=scroll_filter,
        limit=100,       # số lượng mỗi batch
        offset=offset,
        with_payload=True,
        with_vectors=True
    )
    points, offset = scroll_result
    all_points.extend(points)

    if offset is None:
        break

In [11]:
len(all_points)

1449

In [12]:
all_points[:2]

[Record(id='00602903-d9c3-4f7a-a84d-b62fb67309d2', payload={'create_at': '2025-08-26 15:54:15', 'raw_context': 'Dựa vào Xử phạt hành vi vi phạm về thời hạn nộp hồ sơ khai thuế\n1. Phạt cảnh cáo đối với hành vi nộp hồ sơ khai thuế quá thời hạn từ 01 ngày đến 05 ngày và có tình tiết giảm nhẹ.\n2. Phạt tiền từ 2.000.000 đồng đến 5.000.000 đồng đối với hành vi nộp hồ sơ khai thuế quá thời hạn từ 01 ngày đến 30 ngày, trừ trường hợp quy định tại khoản 1 Điều này.\n3. Phạt tiền từ 5.000.000 đồng đến 8.000.000 đồng đối với hành vi nộp hồ sơ khai thuế quá thời hạn quy định từ 31 ngày đến 60 ngày.\n4. Phạt tiền từ 8.000.000 đồng đến 15.000.000 đồng đối với một trong các hành vi sau đây:\na) Nộp hồ sơ khai thuế quá thời hạn quy định từ 61 ngày đến 90 ngày;\nb) Nộp hồ sơ khai thuế quá thời hạn quy định từ 91 ngày trở lên nhưng không phát sinh số thuế phải nộp;\nc) Không nộp hồ sơ khai thuế nhưng không phát sinh số thuế phải nộp;\nd) Không nộp các phụ lục theo quy định về quản lý thuế đối với doanh

In [17]:
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [20]:
llm = ChatOllama(
    model="llama3.1:8b",
    temperature=0.1,  # Low temperature for more consistent answers
)

In [48]:
qa_generate_prompt_tmpl = PromptTemplate(
    template="""
<document>
{context_str}
</document>

Nhiệm vụ: tạo {num_questions_per_chunk} câu hỏi dựa hoàn toàn trên nội dung trong <document>, không dùng kiến thức ngoài.

Yêu cầu:
- Câu hỏi tiếng việt.
- Ngôn ngữ và ngữ điệu tự nhiên
- Không giải thích, chỉ viết câu hỏi.
- Xuất ra đúng {num_questions_per_chunk} câu, mỗi câu một dòng, không số thứ tự.

""", input_variables=["quenum_questions_per_chunkstion", "context_str"])


In [49]:

# Create synthetic dataset chain
qa_generate_chain = (
    {"num_questions_per_chunk": RunnablePassthrough(), "context_str": RunnablePassthrough()}
    | qa_generate_prompt_tmpl
    | llm
    | StrOutputParser()
)

In [51]:
import tqdm

num_questions_per_chunk = 4
all_queries = {}
all_relevant_docs = {}

for point in tqdm.tqdm(all_points):
    doc_id = point.id
    context_str = point.payload['raw_context']
    
    query = qa_generate_prompt_tmpl.format(
        context_str=context_str, num_questions_per_chunk=num_questions_per_chunk
    )
    try:
        response = llm.invoke(query)
    except Exception as e:
        print(f"Error generating questions for document {doc_id}: {e}")
    
    questions = [q.strip() for q in str(response.content).strip().split('\n') if q.strip()]
    num_questions_generated = len(questions)

    queries = {}
    relevant_docs = {}

    for i, question in enumerate(questions):
        question_id = f"{doc_id}-qa_{i}"
        question = re.sub(r'^\s*[\d]+\.\s*', '', question).strip()
        queries[question_id] = question
        relevant_docs[question_id] = [doc_id]

        all_queries |= queries
        all_relevant_docs |= relevant_docs

  0%|          | 0/1449 [00:00<?, ?it/s]

100%|██████████| 1449/1449 [3:50:54<00:00,  9.56s/it]  


In [62]:
import json
import warnings

def save_qa_dataset_to_json(all_queries: dict, all_relevant_docs: dict, all_documents: dict, output_file: str):
    """
    Lưu đối tượng EmbeddingQAFinetuneDataset vào file JSON.
    
    Args:
        all_queries (dict): Dictionary chứa các câu hỏi.
        all_relevant_docs (dict): Dictionary chứa các tài liệu liên quan.
        output_file (str): Tên file JSON để lưu.
    """
    try:
        dataset_dict = {
            "queries": all_queries,
            "relevant_docs": {key: list(value) for key, value in all_relevant_docs.items()},
            "corpus": all_documents
        }
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(dataset_dict, f, ensure_ascii=False, indent=4)
        
        print(f"Dataset đã được lưu thành công vào file: {output_file}")
    except Exception as e:
        warnings.warn(f"Lỗi khi lưu dataset vào file JSON: {e}")

In [63]:
save_qa_dataset_to_json(all_queries=all_queries, 
                        all_relevant_docs=all_relevant_docs, 
                        all_documents=all_documents, 
                        output_file="qa_dataset.json")

Dataset đã được lưu thành công vào file: qa_dataset.json
